In [1]:
  #!/usr/bin/env python
#Numcosmo import
try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm


# General imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.table import Table
import pandas as pd

# DC2 catalog-related imports
import FoFCatalogMatching
import GCRCatalogs
from GCR import GCRQuery

Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

In [2]:
#A list with all DC2 catalogs available

#GCRCatalogs.get_available_catalog_names(include_default_only=False)

In [3]:
#We are loading the cosmoDC2 catalalog with photoz
cosmodc2_small_cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image_with_photozs_v1')

#Taking some important information about the fiducial cosmological model and survey region
cosmo_dc2 = cosmodc2_small_cat.cosmology
sky_area = cosmodc2_small_cat.sky_area

#Listing all quantities in the choosen catalog
#cosmodc2_small_cat.list_all_quantities()

In [16]:
z_min = 0
z_max = 1
M_min = 3.e14
M_max = 1.e15
data = cosmodc2_small_cat.get_quantities(['halo_mass','redshift_true','photoz_mode','photoz_mean','photoz_mask','ra','dec'],
                                   native_filters=['healpix_pixel==9816'] )



photoz_mask = data['photoz_mask']
halo_mass = data['halo_mass'][photoz_mask]
redshift = data['redshift_true'][photoz_mask]
photoz_mean = data['photoz_mean']
photoz_mode = data['photoz_mode']
ra = data['ra'][photoz_mask]
dec = data['dec'][photoz_mask]

dm_data = {'halo_mass': halo_mass, 'z_true': redshift,'photoz_mode':photoz_mode ,'photoz_mean':photoz_mean, 'ra':ra,'dec':dec}
dm_data = pd.DataFrame(dm_data)


mass_min_mask = (M_min<=dm_data['halo_mass'])
mass_max_mask = (M_max >=dm_data['halo_mass'])

z_mask_min = (z_min <=dm_data['z_true'])
z_mask_max = (z_max >=dm_data['z_true'])

mask =  z_mask_min & z_mask_max & mass_max_mask & mass_min_mask 

clusters = dm_data[mask]




N_cl = len(clusters)
print(f'There are {N_cl} halos with Mfof= [{M_min:.2e},{M_max:.2e}] Msun and z=[{z_min},{z_max}] in {cosmodc2_small_cat.sky_area} deg2')
display(clusters)



There are 1403 halos with Mfof= [3.00e+14,1.00e+15] Msun and z=[0,1] in 439.78987 deg2


,halo_mass,z_true,photoz_mode,photoz_mean,ra,dec
26758,7.086487e+14,0.183953,0.235,0.225346,67.980065,-37.137027
26759,7.086487e+14,0.183953,0.355,0.372496,68.018295,-37.150220
26760,7.086487e+14,0.183953,0.235,0.228310,68.137374,-37.112022
26761,7.086487e+14,0.183953,0.225,0.220253,68.060978,-37.151351
26762,7.086487e+14,0.183953,0.185,0.193123,68.006068,-37.141805
...,...,...,...,...,...,...
741360,3.351987e+14,0.951143,1.105,1.172283,67.578294,-36.423048
741361,3.351987e+14,0.951143,0.925,0.996368,67.538850,-36.419207
741362,3.351987e+14,0.951143,0.985,0.998936,67.564681,-36.390525
741363,3.351987e+14,0.951143,0.815,0.825769,67.555742,-36.403128


In [ ]:
fig = plt.figure(figsize=(18,8))
fig = plt.subplot(121)
plt.scatter(clusters['z_true'],clusters['photoz_mode'],s =5)
plt.show()
fig = plt.figure(figsize=(18,8))
fig = plt.subplot(122)
plt.scatter(clusters['z_true'],clusters['photoz_mean'],s =5)